This file is used to train risk models. We train 5 risk models for 5 iterations.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

notebook_dir = os.getcwd()
print(notebook_dir)
os.chdir('./code')
from data_preprocess import read_rdata_to_df, stratified_sample
from generate_risk_model import RiskModelGenerator
os.chdir(notebook_dir)


In [ ]:
feature_data_blr = pd.read_csv('data/accord_blr.csv')
feature_data_f24 = pd.read_csv('data/accord_blr.csv')
outcome_data = pd.read_csv('data/accord_outcomes.csv')

df = pd.merge(outcome_data, feature_data_blr, on='MaskID', how='inner')
df.drop(columns=["MaskID","Visit"], inplace=True)
int_cols = ['cvd','female', 'black', 'cvd_hx_baseline', 'smoke',  'bprx', 'statin']
df[int_cols] = df[int_cols].apply(lambda x: x.astype(int))

In [ ]:
print(df.shape)
print(df.columns.to_list())
df.head()

In [ ]:
# Split the data into training and testing sets (70-30 split)
from sklearn.model_selection import train_test_split

X = df.drop('cvd', axis=1)
y = df['cvd']

# this testing set is the validation set used to check the auc of all fitted risk models
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))
print("\nClass distribution:")
print("Training set:", dict(zip(*np.unique(y_train, return_counts=True))))
print("Testing set:", dict(zip(*np.unique(y_test, return_counts=True))))

In [ ]:
X_train.shape
y_train.shape

In [ ]:
# train 50 iterations, each time using 20% of the training data
n_models = 50
for i in range(1, n_models+1):    
    X_train_model, _, y_train_model, _ = train_test_split(
        X_train, y_train, test_size=0.9, random_state=42+i, stratify=y_train
    )

    model_generator = RiskModelGenerator()
    model_generator.train_models(X_train_model, y_train_model)

In [ ]:
import joblib
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Dictionary to store results
results = {}

# Model names and their display names for the plot
model_names = {
    'logistic_regression': 'Logistic Regression',
    'random_forest': 'Random Forest',
    'xgboost': 'XGBoost',
    'lightgbm': 'LightGBM',
    'svm': 'SVM'
}

# Set up the subplots
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.ravel()

# Colors for different runs
run_colors = ['blue', 'green', 'red', 'purple', 'orange']

# Load models and calculate AUC for each model type
for idx, (model_name, model_display_name) in enumerate(model_names.items()):
    ax = axes[idx]
    
    # Plot random classifier line
    ax.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.500)', alpha=0.5)
    
    # Process each run for this model
    for run in range(1, 6):
        run_dir = f'risk_models/run_{run:03d}'
        try:
            # Load the model
            model_path = os.path.join(run_dir, model_name, f'{model_name}.joblib')
            model = joblib.load(model_path)
            
            # Get predictions
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            
            # Calculate ROC curve and AUC
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            roc_auc = auc(fpr, tpr)
            
            # Store results
            key = f"{model_display_name} (Run {run})"
            results[key] = roc_auc
            
            # Plot ROC curve
            ax.plot(fpr, tpr, 
                   label=f'Run {run} (AUC = {roc_auc:.3f})',
                   color=run_colors[run-1],
                   alpha=0.8)
            
        except Exception as e:
            print(f"Error loading {model_name} from run {run}: {str(e)}")
    
    # Customize each subplot
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'{model_display_name} ROC Curves')
    ax.grid(True)
    ax.legend(loc='lower right')

# Remove the empty subplot
axes[-1].remove()

# Adjust layout
plt.tight_layout()
plt.show()

# Print sorted results grouped by model type
print("\nAUC Scores by Model Type:")
for model_name, model_display_name in model_names.items():
    print(f"\n{model_display_name}:")
    model_results = {k: v for k, v in results.items() if model_display_name in k}
    for key, value in sorted(model_results.items(), key=lambda x: x[1], reverse=True):
        print(f"{key}: {value:.3f}")